In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from scipy.stats import norm
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity

from citableclass import Citable
from corpussearch import search

# Load data

n200  = number of iterations steps  
th10  = allowed std err for linear regression   
sig0.2 = observer deviation from start point  
start0.5 = observer start point  

In [18]:
cite1 = Citable("values_n200_th10_sig0.2_start0.5.json", formats="local", project="kreisgrabenanlage")
cite2 = Citable("values_n500_th15_sig0.2_start0.5.json", formats="local", project="kreisgrabenanlage")
cite3 = Citable("values_n1500_th10_sig0.3_start0.5.json", formats="local", project="kreisgrabenanlage")
cite4 = Citable("values_n2500_th5_sig0.4_start0.5.json", formats="local", project="kreisgrabenanlage")
df = cite2.digitalresource()
df.head()

,gate,object,latitude,longitude,azimuth,prob (az),bins (az),mean azimuth,sigma azimuth,declination,prob (dec),bins (dec),mean declination,sigma declination,number elements
0,1,QuedlinburgI,51.813308,11.178354,"[325.82622, 316.26734, 305.88034, 322.31561, 3...","[0.0018073765, 0.0032220032000000003, 0.005435...","[298.87753, 300.740426, 302.603322, 304.466218...",319.336278,7.932386,"[30.76257, 26.53302, 21.24389, 29.29181, 27.38...","[0.0018407328000000001, 0.0035101353000000003,...","[17.37146, 18.2242875, 19.077115, 19.9299425, ...",27.737216,3.623801,5
1,2,QuedlinburgI,51.813308,11.178354,"[354.41378, 354.7188, 325.3262, 349.86257, 343...","[3.86849e-05, 9.62386e-05, 0.0002247627, 0.000...","[314.50154, 316.7760595, 319.050579, 321.32509...",348.455924,9.049923,"[37.97298, 37.99564, 30.55941, 37.48649, 36.25...","[3.13e-08, 1.7460000000000001e-07, 8.802000000...","[25.67921, 26.304584, 26.929958, 27.555332, 28...",36.773284,1.981355,1
2,3,QuedlinburgI,51.813308,11.178354,"[33.73201, 30.38277, 28.81974, 19.302, 42.3135...","[0.0042385883, 0.0066988568, 0.010056515200000...","[11.05182, 13.2518665, 15.451913, 17.6519595, ...",31.734135,9.701840,"[30.94024, 32.23021, 32.79622, 35.69533, 27.20...","[0.0016354216, 0.0030909080000000003, 0.005544...","[20.74034, 21.571104, 22.401868, 23.232632, 24...",31.286241,3.636081,2
3,5,QuedlinburgI,51.813308,11.178354,"[210.28804, 183.19093, 197.34312, 209.29807, 2...","[0.0052987368, 0.0081840125, 0.0117609403, 0.0...","[180.03181, 184.380746, 188.729682, 193.078618...",208.422774,16.194851,"[-32.26518, -38.11685, -36.16547, -32.62552, -...","[0.0372032105, 0.04895783, 0.0589042854, 0.064...","[-38.18668, -36.3697285, -34.552777, -32.73582...",-31.711064,6.069680,1
4,6,QuedlinburgI,51.813308,11.178354,"[276.62348, 288.42604, 282.63551, 273.00917, 3...","[0.0162189614, 0.023222578100000002, 0.0292907...","[270.02515, 274.359896, 278.694642, 283.029388...",284.463660,12.173372,"[4.08916, 11.26862, 7.77225, 1.85982, 21.46735...","[0.0261028799, 0.0358249365, 0.0454668862, 0.0...","[0.015550000000000001, 1.920418, 3.825286, 5.7...",8.674223,6.809249,1


## Auxiliary functions

In [19]:
# auxiliary to simplify normal distributions for n data points
def ndi(mu,sd,n):
    b=np.random.normal(mu,sd,n)
    return(b)

In [26]:
from math import *

def az2dec(phi,A):
    rad_phi=radians(phi)
    rad_A=radians(A)
    a=degrees(asin(cos(rad_phi)*cos(rad_A))) # for alt=0
    return(round(a,5))

def hor_to_equ1(x):
    return(az2dec(float(x["latitude"]),float(x["mean azimuth"])))

In [28]:
df["rev mean decl"]=df.apply(hor_to_equ1,axis=1)
df.head()

,gate,object,latitude,longitude,azimuth,prob (az),bins (az),mean azimuth,sigma azimuth,declination,prob (dec),bins (dec),mean declination,sigma declination,number elements,c
0,1,QuedlinburgI,51.813308,11.178354,"[325.82622, 316.26734, 305.88034, 322.31561, 3...","[0.0018073765, 0.0032220032000000003, 0.005435...","[298.87753, 300.740426, 302.603322, 304.466218...",319.336278,7.932386,"[30.76257, 26.53302, 21.24389, 29.29181, 27.38...","[0.0018407328000000001, 0.0035101353000000003,...","[17.37146, 18.2242875, 19.077115, 19.9299425, ...",27.737216,3.623801,5,27.96638
1,2,QuedlinburgI,51.813308,11.178354,"[354.41378, 354.7188, 325.3262, 349.86257, 343...","[3.86849e-05, 9.62386e-05, 0.0002247627, 0.000...","[314.50154, 316.7760595, 319.050579, 321.32509...",348.455924,9.049923,"[37.97298, 37.99564, 30.55941, 37.48649, 36.25...","[3.13e-08, 1.7460000000000001e-07, 8.802000000...","[25.67921, 26.304584, 26.929958, 27.555332, 28...",36.773284,1.981355,1,37.28065
2,3,QuedlinburgI,51.813308,11.178354,"[33.73201, 30.38277, 28.81974, 19.302, 42.3135...","[0.0042385883, 0.0066988568, 0.010056515200000...","[11.05182, 13.2518665, 15.451913, 17.6519595, ...",31.734135,9.701840,"[30.94024, 32.23021, 32.79622, 35.69533, 27.20...","[0.0016354216, 0.0030909080000000003, 0.005544...","[20.74034, 21.571104, 22.401868, 23.232632, 24...",31.286241,3.636081,2,31.72210
3,5,QuedlinburgI,51.813308,11.178354,"[210.28804, 183.19093, 197.34312, 209.29807, 2...","[0.0052987368, 0.0081840125, 0.0117609403, 0.0...","[180.03181, 184.380746, 188.729682, 193.078618...",208.422774,16.194851,"[-32.26518, -38.11685, -36.16547, -32.62552, -...","[0.0372032105, 0.04895783, 0.0589042854, 0.064...","[-38.18668, -36.3697285, -34.552777, -32.73582...",-31.711064,6.069680,1,-32.93618
4,6,QuedlinburgI,51.813308,11.178354,"[276.62348, 288.42604, 282.63551, 273.00917, 3...","[0.0162189614, 0.023222578100000002, 0.0292907...","[270.02515, 274.359896, 278.694642, 283.029388...",284.463660,12.173372,"[4.08916, 11.26862, 7.77225, 1.85982, 21.46735...","[0.0261028799, 0.0358249365, 0.0454668862, 0.0...","[0.015550000000000001, 1.920418, 3.825286, 5.7...",8.674223,6.809249,1,8.88268


# Simulation
## Generation of mixed distribution

## Parameter

In [ ]:
# no of gates, gaussian distribution of directions
n_gates= 4
# mean values
#value:  -30.361514640687762 var 9.488898890923112
#value:  37.18086067610277 var 14.458853132679794
#value:  12.767138961255649 var 4.543496345526992
#value:  -8.563945763567334 var 5.7156606842991975
#value:  -37.42626719012814 var 8.738291591935992

mu=(-37,-30,-8,12,37,45,26,19)
# std. variation
sd=1
# additional noise
noise_bmg = 0.5
# number of simulated directions
n=250

### Mixed distribution of simulated data points

In [ ]:
# mixed distribution from simulation
a = [ndi(mu[i],sd,n) for i in range(n_gates)]
bmg=np.concatenate(a)
# simulated data for mixed distribution
data = np.random.normal(bmg, noise_bmg)[:, np.newaxis]

In [ ]:
sns.distplot(data);

In [ ]:
n_components = np.arange(1, 21)
models = [GaussianMixture(n, covariance_type='spherical').fit(data) for n in n_components]

In [ ]:
plt.figure(figsize=(9,7))
plt.plot(n_components, [m.bic(data) for m in models], label='BIC')
plt.plot(n_components, [m.aic(data) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

# Site data

In [ ]:
# minimal number of defining features pro gates
number=0
# upper bound for sigma
sigma=4
# minimum declination
mdec=-50
# max azimuth
maxaz=380
# list of empirical deviations sd from selected gates
sel_df=df[["mean declination","sigma declination"]][(df["mean azimuth"] < maxaz) & (df["mean declination"] > mdec) & (df["number elements"] > number) & (df["sigma declination"]< sigma)]
sel_df.head()

In [ ]:
l1=sel_df["mean declination"].tolist()
l2=sel_df["sigma declination"].tolist()
print(len(l1),len(l2))

In [ ]:
# mixed distribution from simulation
b = tuple([ndi(l1[i],l2[i],n) for i in range(len(l1))])
bmg=np.concatenate(b)
# simulated data for mixed distribution
data = np.random.normal(bmg, noise_bmg)[:, np.newaxis]

In [ ]:
plt.hist(data, bins='auto', alpha=.6, density=True)

In [ ]:
sns.distplot(data);

In [ ]:
gmm = GaussianMixture(n_gates,covariance_type='spherical')
gmm.fit(data)
x = np.linspace(-45, 45, 3 * 2500 + 1)
plt.figure(figsize=(9,7))
plt.hist(data, bins='auto', alpha=.3, density=True, log=False)

for i in np.arange(n_gates):
    mu=gmm.means_[i,0]
    var=gmm.covariances_[i]
    print("value: ", mu,"var",var)
    plt.vlines(mu, ymin=0, ymax=0.05, label='Fitted Means')
    plt.plot(x, norm.pdf(x, mu, np.sqrt(var)))
    plt.legend()
    plt.title('Gaussian Mixture Model')

### Anzahl der Gaussians entnommen aus AIC/BIC 

The optimal number of clusters is the value that minimizes the AIC or BIC, depending on which approximation we wish to use. The AIC tells us that our choice of 16 components above was probably too many: around 8-12 components would have been a better choice. As is typical with this sort of problem, the BIC recommends a simpler model.

Notice the important point: this choice of number of components measures how well GMM works as a density estimator, not how well it works as a clustering algorithm. I'd encourage you to think of GMM primarily as a density estimator, and use it for clustering only when warranted within simple datasets.

In [ ]:
n_components = np.arange(1, 21)
models = [GaussianMixture(n, covariance_type='spherical').fit(data) for n in n_components]

In [ ]:
plt.figure(figsize=(9,7))
plt.plot(n_components, [m.bic(data) for m in models], label='BIC')
plt.plot(n_components, [m.aic(data) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

In [ ]:
# best n gates according to BIC
n_gates=5
gmm = GaussianMixture(n_gates,covariance_type='spherical')
gmm.fit(data)
x = np.linspace(-45, 45, 3 * 2500 + 1)
plt.figure(figsize=(9,7))
plt.hist(data, bins='auto', alpha=.3, density=True, log=False)

for i in np.arange(n_gates):
    mu=gmm.means_[i,0]
    var=gmm.covariances_[i]
    print("value: ", mu,"var",var)
    plt.vlines(mu, ymin=0, ymax=0.05, label='Fitted Means')
    plt.plot(x, norm.pdf(x, mu, np.sqrt(var)))
    plt.legend()
    plt.title('Gaussian Mixture Model')

# 1d GMM Modeling Literature

https://km3pipe.readthedocs.io/en/latest/auto_examples/stats/plot_fitting_dists.html

B.W. Silverman, “Density Estimation for Statistics and Data Analysis”
Hastie, Tibshirani and Friedman, “The Elements of Statistical Learning: Data Mining, Inference, and Prediction”, Springer (2009)
Liu, R., Yang, L. “Kernel estimation of multivariate cumulative distribution function.” Journal of Nonparametric Statistics (2008)

In [ ]:
dens = sm.nonparametric.KDEUnivariate(data)
dens.fit()

kde_sm = dens.evaluate(x)
plt.fill_between(x, kde_sm, alpha=.5, label='KDE')
plt.hist(data, bins='auto', alpha=.3, density=True)